## check the contents in laptime dataset

base:   


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import ipdb; 

In [20]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#check for car12
def check_laptimedata(_laptime_data, check_carno = 12, train_ratio=0):
    
    run_ts = COL_RANK
    totalTSCnt = 0
    totalTSLen = 0        
    
    for _data in _laptime_data:
        _train = []
        _test = []

        if events[_data[0]] == _test_event:
            test_mode = True
        else:
            test_mode = False

        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        if train_len == 0:
            #use global train_len
            train_len = _train_len

        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10

        print(f'after ====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()

            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]

            # remove short ts
            totallen = rec.shape[1]

            totalTSCnt += 1
            totalTSLen += totallen

            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                

            carno = _data[1][rowid]
            #carid = global_carids[_data[1][rowid]]

            if carno == check_carno:
                #show content in rec

                #rank
                print('rank:',rec[COL_RANK,:] )

                #pitlaps
                pits = np.where(rec[COL_LAPSTATUS,:]==1)
                print('pits:', pits)
                
                
def adjustrank_laptimedata(_laptime_data):
    
    run_ts = COL_RANK
    
    for _data in _laptime_data:
        _train = []
        _test = []

        if events[_data[0]] == _test_event:
            test_mode = True
        else:
            test_mode = False

        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        if train_len == 0:
            #use global train_len
            train_len = _train_len

        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10

        print(f'after ====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()

            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]

            # remove short ts
            totallen = rec.shape[1]

            totalTSCnt += 1
            totalTSLen += totallen

            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                

            carno = _data[1][rowid]
            #carid = global_carids[_data[1][rowid]]

            if carno == check_carno:
                #show content in rec

                #rank
                print('rank:',rec[COL_RANK,:] )

                #pitlaps
                pits = np.where(rec[COL_LAPSTATUS,:]==1)
                print('pits:', pits)                
                
def diff_laptimedata(lapdata1, lapdata2):
    """
    compare two laptime datasets
    """
    
    def get_events(lapdata):
        return [events[x[0]] for x in lapdata]
    
    
    #1. check events
    events1 = get_events(lapdata1)
    events2 = get_events(lapdata1)

    events_cnt = min(len(events1, events2))
    
    print(events1, events2)
    
    #2. check features
    run_ts = COL_RANK
    
    for _data in _laptime_data:
        _train = []
        _test = []

        if events[_data[0]] == _test_event:
            test_mode = True
        else:
            test_mode = False

        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        if train_len == 0:
            #use global train_len
            train_len = _train_len

        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10

        print(f'after ====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()

            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]

            # remove short ts
            totallen = rec.shape[1]

            totalTSCnt += 1
            totalTSLen += totallen

            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                

            carno = _data[1][rowid]
            #carid = global_carids[_data[1][rowid]]

            if carno == check_carno:
                #show content in rec

                #rank
                print('rank:',rec[COL_RANK,:] )

                #pitlaps
                pits = np.where(rec[COL_LAPSTATUS,:]==1)
                print('pits:', pits)                


In [9]:
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
COL_CAUTION_LAPS_INSTINT=5
COL_LAPS_INSTINT= 6
COL_ELAPSED_TIME= 7
COL_LAP2NEXTPIT = 8
#_featureCnt = 9

# added new features
COL_LEADER_PITCNT = 9

FEATURE_STATUS = 2
FEATURE_PITAGE = 4
FEATURE_LEADERPITCNT = 8

#inlap status = 
# 0 , no inlap
# 1 , set previous lap
# 2 , set the next lap
_inlap_status = 0

#
# featuremode in [FEATURE_STATUS, FEATURE_PITAGE]:
#
_feature_mode = FEATURE_LEADERPITCNT
_featureCnt = 9

#
# training parameters
#
freq = "1min"
_train_len = 40
prediction_length = 2

context_ratio = 0.
context_length =  40
contextlen = context_length

dataset='rank'
_run_ts = COL_RANK

_test_event = 'Indy500-2018'
year = '2018'

#
# string map
#
inlapstr = {0:'noinlap',1:'inlap',2:'outlap'}
featurestr = {FEATURE_STATUS:'nopitage',FEATURE_PITAGE:'pitage',FEATURE_LEADERPITCNT:'leaderpitcnt'}
weightstr = {True:'weighted',False:'noweighted'}
catestr = {True:'cate',False:'nocate'}

#
# input data parameters
#
years = ['2013','2014','2015','2016','2017','2018','2019']
events = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(events)}
dbid = f'Indy500_{years[0]}_{years[-1]}_v{_featureCnt}_p{_inlap_status}'
_dataset_id = '%s-%s'%(inlapstr[_inlap_status], featurestr[_feature_mode])

# standard output file names
LAPTIME_DATASET = f'laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'stagedata-{dbid}.pickle' 
EVALUATION_RESULT_DF = f'evaluation_result_d{dataset}.csv'
LONG_FORECASTING_DFS = f'long_forecasting_dfs_d{dataset}.pickle'
FORECAST_FIGS_DIR = f'forecast-figs-d{dataset}/'  

In [23]:
outdir = 'test/'
outputRoot = outdir
os.makedirs(outdir, exist_ok=True)

_task_dir = f'{outdir}/'

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
dbname = _task_dir + f'gluontsdb-{dataset}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{_test_event}-indy-{year}.pickle'
laptimedb = _task_dir + f'gluontsdb-{dataset}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{_test_event}-indy-{year}-newlaptimedata.pickle'

print('Load Laptime Dataset:',laptimedb)
with open(_task_dir + LAPTIME_DATASET, 'rb') as f:
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 

print('Load New Laptime Dataset:',laptimedb)
with open(laptimedb, 'rb') as f:
    prepared_laptimedata = pickle.load(f, encoding='latin1') 

Load Laptime Dataset: test//gluontsdb-rank-oracle-noip-noeid-all-all-f1min-t2-rIndy500-2018-indy-2018-newlaptimedata.pickle
Load New Laptime Dataset: test//gluontsdb-rank-oracle-noip-noeid-all-all-f1min-t2-rIndy500-2018-indy-2018-newlaptimedata.pickle


In [24]:
check_laptimedata(prepared_laptimedata, check_carno=12)                

after ====event:Indy500-2013, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
rank: [ 4.  4.  4.  4.  4.  4.  5.  7.  7.  7.  7.  7.  7.  8.  8.  8.  8.  7.
  7.  7.  7.  7.  7.  7.  7.  7.  7.  6.  5.  2.  0.  0.  4. 12. 11. 11.
 11.  9. 10. 10. 10.  9.  9.  9.  9.  9.  7.  8.  9.  9.  8.  8.  8.  8.
  8.  8.  7.  7.  4.  4.  4.  4.  4.  4.  4.  4.  3.  3.  3.  3.  3.  2.
  2.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 22. 24.
 17. 15.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  7.  9.  9.  9.  9.  9.
  9. 10. 10.  9.  9.  9.  9.  9.  8.  8.  8.  8. 16. 16. 15. 12. 10. 10.
 12. 12. 13. 13. 13. 13. 13. 12. 12. 11. 11. 11. 12. 12. 12. 12. 11. 11.
 11. 11. 11. 11. 11. 11. 10.  8.  4.  7. 16. 16. 15. 14. 14. 15. 15. 15.
 15. 15. 15. 15. 14. 12. 12. 12. 12. 12. 11. 11. 10. 10. 10.  7.  7.  6.
  4.  3.  2.  2.  4. 11. 11. 11. 10. 11. 11. 11. 11. 12. 11. 12. 19. 18.
 18. 18.]
pits: (array([ 32,  57,  88, 120, 153, 184, 195]),)
after ====event:Indy500-2014

In [22]:
check_laptimedata(laptime_data, check_carno=12)

after ====event:Indy500-2013, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
a short ts: carid=4，len=3
a short ts: carid=6，len=34
rank: [ 5.  5.  5.  5.  5.  5.  6.  8.  8.  8.  8.  8.  8.  9.  9.  9.  9.  8.
  8.  8.  8.  8.  8.  8.  8.  8.  8.  7.  6.  3.  1.  1.  5. 13. 12. 12.
 12. 10. 11. 11. 11. 10. 10. 10. 10. 10.  8.  9. 10. 10.  9.  9.  9.  9.
  9.  9.  8.  8.  5.  5.  5.  5.  5.  5.  5.  5.  4.  4.  4.  4.  4.  3.
  3.  3.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 23. 25.
 18. 16.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  8. 10. 10. 10. 10. 10.
 10. 11. 11. 10. 10. 10. 10. 10.  9.  9.  9.  9. 17. 17. 16. 13. 11. 11.
 13. 13. 14. 14. 14. 14. 14. 13. 13. 12. 12. 12. 13. 13. 13. 13. 12. 12.
 12. 12. 12. 12. 12. 12. 11.  9.  5.  8. 17. 17. 16. 15. 15. 16. 16. 16.
 16. 16. 16. 16. 15. 13. 13. 13. 13. 13. 12. 12. 11. 11. 11.  8.  8.  7.
  5.  4.  3.  3.  5. 12. 12. 12. 11. 12. 12. 12. 12. 13. 12. 13. 20. 19.
 19. 19.]
pits: (array([ 32,  57,  88